In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pinocchio as pin
from scipy.optimize import least_squares
from scipy.special import huber
from projectyl.utils.properties import ELBOW, SHOULDER, WRIST
from projectyl.utils.arm import plot_optimization_curves

from projectyl.dynamics.inverse_kinematics import build_arm_model
from projectyl.utils.arm import interactive_replay_sequence
from projectyl.utils.io import Dump
%load_ext autoreload
%autoreload 2
global_params = {}
build_arm_model(global_params, headless=False)
arm_robot = global_params["arm"]
viz = global_params["viz"]  

In [ ]:
arm_robot.model.createData()
viz.display(arm_robot.q0)

TESTS arm moving, random constant acceleration

In [ ]:
q = arm_robot.q0.copy()
vq = np.zeros(arm_robot.model.nv)
aq = 5 * np.random.rand(arm_robot.model.nv)
viz.display(q)
DT = 1e-2

T = 30

rec_gt_sol = []
rec_gt_p = []

shoulder_frame_id = arm_robot.model.getFrameId(SHOULDER)
elbow_frame_id = arm_robot.model.getFrameId(ELBOW)
wrist_frame_id = arm_robot.model.getFrameId(WRIST)

In [ ]:
for _ in range(T):
    # Iterative forward dynamics
    vq += aq * DT
    q = pin.integrate(arm_robot.model, q, vq * DT)
    tauq = pin.rnea(arm_robot.model, arm_robot.data, q, vq, aq)
    
    # Retrieve 3D points (forward kinematics)
    pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)
    shoulder_p = arm_robot.data.oMf[shoulder_frame_id].translation
    elbow_p = arm_robot.data.oMf[elbow_frame_id].translation
    end_effector_p = arm_robot.data.oMf[wrist_frame_id].translation

    viz.display(q)
    # Store ground truth "solutions" (q, vq, tau)
    xuc = np.concatenate([q, vq, tauq])
    rec_gt_sol.append(xuc)
    # Store ground truth data (p)
    p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
    rec_gt_p.append(p)
    time.sleep(1e-2)

In [ ]:
np.array(rec_gt_p).shape

In [ ]:
q.shape # 4 +1 DOF elbow
vq.shape # 4 =  3(axis angle 3D ~  log map of quaternion 4D ) + 1 DOF elbow
tauq.shape # 4 = 3 + 1 
xuc.shape # 5 + 4 + 4 = 13  = position + velocity + torque

In [ ]:
gt_sol = np.vstack(rec_gt_sol)
gt_p = np.vstack(rec_gt_p)
gt_sol.shape, gt_p.shape # T=180 step, 13 "xuc" and 9 3D positions
# sol = solution...
# p = position = oberserved data...

# Cost terms definitions

In [ ]:
def get_pose_velocity_from_state(tq_normalized, tvq, T):
    # Forward pass
    rec_p = []
    rec_v = []

    for t in range(T):
        q = tq_normalized[t]
        vq = tvq[t]
        # (forward kinematics)
        pin.forwardKinematics(arm_robot.model, arm_robot.data, q, vq)
        pin.updateFramePlacements(arm_robot.model, arm_robot.data)
        # Predicted 3D points -> 3x3 points
        shoulder_p = arm_robot.data.oMf[shoulder_frame_id].translation
        elbow_p = arm_robot.data.oMf[elbow_frame_id].translation
        end_effector_p = arm_robot.data.oMf[wrist_frame_id].translation
        # Predicted 3D velocities -> ingnore shoulder 2x3 points
        shoulder_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, shoulder_frame_id, pin.ReferenceFrame.WORLD).linear
        elbow_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, elbow_frame_id, pin.ReferenceFrame.WORLD).linear
        end_effector_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, wrist_frame_id, pin.ReferenceFrame.WORLD).linear

        p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
        rec_p.append(p)

        v = np.concatenate([shoulder_v, elbow_v, end_effector_v])
        rec_v.append(v)

    tp = np.vstack(rec_p)
    tv = np.vstack(rec_v)

    ta = (tv[1:] - tv[:-1]) / DT

    return tp, tv, ta

In [ ]:
def diff_3D(tp, tp_observed):
    diff = tp - tp_observed
    diff = diff.flatten()

    return diff

In [ ]:
def smooth_velocity_acceleration(tv, ta):

    rv = tv.flatten()
    ra = ta.flatten()

    return np.concatenate([rv, ra])

In [ ]:
def smooth_torque(ttauq):
    return ttauq.flatten()

In [ ]:
# If the Lagrange dynamics formulation cannote be totally satisfied,
# one can relax by minimizing the difference between:
# - the Lagrange dynamics torques tau_rec predicited from q, vq and aq.
# - the current predicted torques ttauq
# question: why not also going backward in time ?
def full_body_dynamics(tq_normalized, tvq, taq, ttauq, T):
    
    tau_rec = []

    for t in range(1, T):
        tau = pin.rnea(arm_robot.model, arm_robot.data, tq_normalized[t], tvq[t], taq[t - 1])
        # taq[t - 1] ?

        tau_rec.append(tau)
    
    ttau = np.vstack(tau_rec)

    diff = ttau - ttauq[1:]
    # ttauq[1:] - cannot compute ttau for t=0
    
    diff = diff.flatten()

    return diff

In [ ]:
# Build the cost function
def objective(txuc, tp_observed, T, debug=False) -> np.ndarray:
    txuc_r = txuc.reshape(T, -1)

    tq = txuc_r[:, :arm_robot.model.nq]
    norm_quat = np.linalg.norm(tq[:, :-1], axis=1, keepdims=True)
    tq_normalized = tq.copy()
    tq_normalized[:, :-1] /= norm_quat

    tvq = txuc_r[:, arm_robot.model.nq: arm_robot.model.nq + arm_robot.model.nv]

    taq = (tvq[1:] - tvq[:-1]) / DT

    ttauq = txuc_r[:, arm_robot.model.nq + arm_robot.model.nv:]

    tp, tv, ta = get_pose_velocity_from_state(tq_normalized, tvq, T)

    # DEBUG
    if debug:
        print("3D joint positions", np.linalg.norm(diff_3D(tp, tp_observed)))
        print("Smooth velocity acceleration", np.linalg.norm(smooth_velocity_acceleration(tv, ta)))
        print("Smooth torque", np.linalg.norm(smooth_torque(ttauq)))
        print("Full body dynamics", np.linalg.norm(full_body_dynamics(tq_normalized, tvq, taq, ttauq, T)))

    res = np.concatenate([
        diff_3D(tp, tp_observed),
        0.1 * smooth_velocity_acceleration(tv, ta),
        0.1 * smooth_torque(ttauq),
        10 * full_body_dynamics(tq_normalized, tvq, taq, ttauq, T),
    ])

    return res

In [ ]:
plot_optimization_curves([(gt_sol, "[gt]", "--"),])

In [ ]:
gt_sol[:, :arm_robot.model.nq].shape

In [ ]:
interactive_replay_sequence(
    {"groundtruth":gt_sol[:, :arm_robot.model.nq]},
    viz
)

In [ ]:
# intialized with ground truth data for angular positions
# and 0 for velocity
# and 0 for torques
q = gt_sol[:, :arm_robot.model.nq]

initial = np.concatenate([q, np.zeros((T, 2 * arm_robot.model.nv))], axis=1)

# Optimization

In [ ]:
sol = least_squares(objective, initial.flatten(), args=(gt_p, T)).x

In [ ]:
res = sol.reshape(T, -1)

In [ ]:
interactive_replay_sequence(
    {
        "estimated": res[:, :arm_robot.model.nq],
        "groundtruth": gt_sol[:, :arm_robot.model.nq],
    },
    viz
)

In [ ]:
plot_optimization_curves([
    (gt_sol, "[gt]", "--"),
    (res, "[estimation]", "-"),
    ]
)
plot_optimization_curves([
    (gt_sol, "[gt]", "--"),
    ]
)

plot_optimization_curves([
    (res, "[estimation]", "-"),
    ]
)

In [ ]:
sol.shape

In [ ]:
print(np.linalg.norm(gt_sol.flatten()- sol.flatten()))

In [ ]:
res = sol.reshape(T, -1)

In [ ]:
np.linalg.norm(objective(sol, gt_p, T, True))

In [ ]:
np.linalg.norm(objective(gt_sol.flatten(), gt_p, T, True))

In [ ]:
tq = res[:, :arm_robot.model.nq]
print(tq.shape)
norm_quat = np.linalg.norm(tq[:, :-1], axis=1, keepdims=True)
tq_normalized = tq.copy()
tq_normalized[:, :-1] /= norm_quat

tvq = res[:, arm_robot.model.nq: arm_robot.model.nq + arm_robot.model.nv]


ttauq = res[:, arm_robot.model.nq + arm_robot.model.nv:]

In [ ]:
for t in range(T):

    viz.display(tq_normalized[t])

    time.sleep(1e-2)

In [ ]:
for t in range(T):
    viz.display(gt_sol[t, :arm_robot.model.nq])
    time.sleep(1e-2)